In [4]:
%%capture
!python -m pip install abraia
!python -m pip install onnx onnxruntime

import os
if not os.getenv('ABRAIA_ID') and not os.getenv('ABRAIA_KEY'):
    #@markdown <a href="https://abraia.me/console/gallery" target="_blank">Upload and manage your images</a>
    abraia_id = ''  #@param {type: "string"}
    abraia_key = ''  #@param {type: "string"}
    %env ABRAIA_ID=$abraia_id
    %env ABRAIA_KEY=$abraia_key

from abraia import Abraia

multiple = Abraia()

In [ ]:
import torch
import torchvision

dummy_input = torch.randn(1, 3, 224, 224)
model = torchvision.models.alexnet(pretrained=True)
model.eval()

input_names = ["input1"]
output_names = ["output1"]

torch.onnx.export(
    model,
    dummy_input,
    "assets/model.onnx",
    verbose=True,
    input_names=input_names,
    output_names=output_names,
)

In [5]:
import torch
from abraia.torch import load_classes, load_model, read_image, transform

dataset = 'hymenoptera_data'

class_names = load_classes(os.path.join(dataset, 'model_ft.txt'))
model = load_model(os.path.join(dataset, 'model_ft.pt'), class_names)
# model.eval()

img = read_image(os.path.join(dataset, 'dog.jpg'))
input_batch = transform(img).unsqueeze(0)
# TODO: Define export_onnx
torch.onnx.export(model, input_batch, 'model_ft.onnx', export_params=True, opset_version=10, do_constant_folding=True, input_names=['input'], output_names=['output'])
# onnx_model = onnx.load('model_ft.onnx')
# onnx.checker.check_model(onnx_model)

In [6]:
import time
import numpy as np
import onnxruntime as ort
from PIL import Image

def preprocess(img):
    '''The function takes loaded image and returns processed tensor.'''
    img = np.array(img.resize((256, 256))).astype(np.float32)
    #center crop
    rm_pad = (256-224)//2
    img = img[rm_pad:-rm_pad,rm_pad:-rm_pad]
    #normalize by mean + std
    img = (img / 255. - np.array([0.485, 0.456, 0.406])) / np.array([0.229, 0.224, 0.225])
    img = img.transpose((2, 0, 1))
    img = np.expand_dims(img, axis=0)
    return img

def predict(path):
    img = Image.open(path)
    img_batch = preprocess(img)
    outputs = ort_session.run(None, {"input": img_batch.astype(np.float32)})
    a = np.argsort(-outputs[0].flatten())
    results = {}
    for i in a[0:5]:
        results[labels[i]]=float(outputs[0][0][i])
    return results

ort_session = ort.InferenceSession("model_ft.onnx", providers=['CPUExecutionProvider'])

labels = class_names
image_path = "/tmp/hymenoptera_data/dog.jpg"
predict(image_path)

{'dogs': 3.2437400817871094,
 'bees': -1.3845250606536865,
 'ants': -1.5448365211486816,
 'cats': -2.4362499713897705}